In [2]:
import yaml
import time
import requests

In [3]:
def load_cfg(path):
    with open(path, 'r') as f:
        data = yaml.safe_load(f)
    return data

def request_job_from_server(url, url_status, payload):
    job = requests.post(url, json=payload).json()
    job_id = job["id"]
    
    while True:
        resp = requests.get(f"{url_status}/{job_id}", timeout=10)
        resp.raise_for_status()
        status = resp.json()

        if status["status"] in ("success", "failed"):
            return status
        time.sleep(2)

In [1]:
url = 'http://localhost:8000/runs'

payload = {
    'run_type': 'base'
}
run = requests.post(url, json=payload).json()

run_id = run['run_id']
url_exec_jobs = f'{url}/{run_id}/exec-jobs'
url_prepare_jobs = f'{url}/{run_id}/prepare-jobs'

NameError: name 'requests' is not defined

In [100]:
ds_data = load_cfg('fashion_prepare_dataset.yaml')
job = requests.post(f'{url_prepare_jobs}/dataset', json=ds_data)

In [101]:
model_data = load_cfg('prepare_model.yaml')
job = requests.post(f'{url_prepare_jobs}/model', json=model_data)
print(job.json())

{'id': 'b24cfec5382e4a1fa841dd3eff1257f7', 'job_type': 2, 'status': 'pending', 'status_details': None, 'error': None, 'created_at': '2026-02-17T13:48:57.542429+00:00', 'expires_at': '2026-02-17T15:48:57.542470+00:00'}


In [102]:
train_params_data = load_cfg('train.yaml')
job = requests.post(f'{url_prepare_jobs}/train-params', json=train_params_data)

In [103]:
train_data = {
    'exp_name': 'test',
    'run_name': 'test',
    'model_name': 'test'
}
job = requests.post(f'{url_exec_jobs}/train', json=train_data)

In [18]:
url = 'http://localhost:8000/runs'

payload = {
    'run_type': 'post_process'
}
run = requests.post(url, json=payload).json()

run_id = run['run_id']
url_exec_jobs = f'{url}/{run_id}/exec-jobs'
url_prepare_jobs = f'{url}/{run_id}/prepare-jobs'

In [19]:
data = {
    'run_id': 'ce99d5689734423aa79e3dfe258f9b5d'
}

job = requests.post(f'{url_prepare_jobs}/load-run', json=data)

In [182]:
pp_data = load_cfg('pp_cfg.yaml')
job = requests.post(f'{url_prepare_jobs}/post-process', json=pp_data)

In [183]:
data = {
    'exp_name': 'test',
    'run_name': 'pp_test',
    'model_name': 'pp_test'
}
job = requests.post(f'{url_exec_jobs}/final-evaluation', json=data)
job.json()

{'id': '28de93e79383429eb0832f59a6bdfc47',
 'job_type': 8,
 'status': 'pending',
 'status_details': None,
 'error': None,
 'created_at': '2026-02-17T16:42:53.355030+00:00',
 'expires_at': '2026-02-17T18:42:53.355059+00:00'}

In [20]:
requests.get(f'{url}/{run_id}').json()

{'run_id': 'c862349d375a4a6590c8d6029539759d',
 'run_type': 'post_process',
 'state': 'prepare_default_run',
 'jobs': [{'id': '3c9b67a5b0f24f92a051766402e7453a',
   'job_type': 4,
   'status': 'success',
   'status_details': 'Configurations from run are successfully prepared',
   'error': None,
   'created_at': '2026-02-17T21:46:41.431622+00:00',
   'expires_at': '2026-02-17T23:46:41.431656+00:00'}],
 'created_at': '2026-02-17T21:46:38.835117+00:00',
 'updated_at': '2026-02-17T21:47:03.255123+00:00'}

In [9]:
data = {
    "id": 'fashion_mnist'
}
resp = requests.post('http://localhost:8000/data-info/get-dataset-info', json=data)

In [11]:
resp.json()['status_details']

{'description': '',
 'citation': '',
 'homepage': '',
 'license': '',
 'features': {'image': {'mode': None,
   'decode': True,
   'id': None,
   '_type': 'Image'},
  'label': {'names': ['T - shirt / top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'],
   'id': None,
   '_type': 'ClassLabel'}},
 'post_processed': None,
 'supervised_keys': None,
 'builder_name': 'parquet',
 'dataset_name': 'fashion_mnist',
 'config_name': 'fashion_mnist',
 'version': {'version_str': '0.0.0',
  'description': None,
  'major': 0,
  'minor': 0,
  'patch': 0},
 'splits': {'train': {'name': 'train',
   'num_bytes': 31049107.0,
   'num_examples': 60000,
   'shard_lengths': None,
   'dataset_name': None},
  'test': {'name': 'test',
   'num_bytes': 5192560.0,
   'num_examples': 10000,
   'shard_lengths': None,
   'dataset_name': None}},
 'download_checksums': None,
 'download_size': 36106894,
 'post_processing_size': None,
 'dataset

In [7]:
resp = requests.get('http://localhost:8000/mlflow/history')
resp.json()

{'exps': [{'name': 'test', 'url': 'http://localhost:5000/#/experiments/1'},
  {'name': 'Default', 'url': 'http://localhost:5000/#/experiments/0'}]}

In [9]:
resp = requests.get('http://localhost:8000/mlflow/get-exp-runs/test')
resp.json()

{'runs': [{'run_id': 'a3141960405e4497aa11a69072825f41',
   'url': 'http://localhost:5000/#/experiments/1/runs/a3141960405e4497aa11a69072825f41'},
  {'run_id': 'ce99d5689734423aa79e3dfe258f9b5d',
   'url': 'http://localhost:5000/#/experiments/1/runs/ce99d5689734423aa79e3dfe258f9b5d'}]}

In [17]:
mlflow_run_id = 'a3141960405e4497aa11a69072825f41'

resp = requests.get(f'http://localhost:8000/mlflow/{mlflow_run_id}')
resp.json()

{'results': {'val_metrics': {'val_accuracy': 86.98349174587293},
  'val_error_table': [{'id': 5299,
    'is_correct': True,
    'y_true': 0,
    'y_pred': 0},
   {'id': 6353, 'is_correct': True, 'y_true': 3, 'y_pred': 3},
   {'id': 4184, 'is_correct': False, 'y_true': 7, 'y_pred': 9},
   {'id': 4415, 'is_correct': True, 'y_true': 3, 'y_pred': 3},
   {'id': 3715, 'is_correct': True, 'y_true': 4, 'y_pred': 4},
   {'id': 4653, 'is_correct': True, 'y_true': 9, 'y_pred': 9},
   {'id': 5314, 'is_correct': True, 'y_true': 9, 'y_pred': 9},
   {'id': 9444, 'is_correct': True, 'y_true': 7, 'y_pred': 7},
   {'id': 7995, 'is_correct': True, 'y_true': 0, 'y_pred': 0},
   {'id': 3617, 'is_correct': True, 'y_true': 4, 'y_pred': 4},
   {'id': 8857, 'is_correct': True, 'y_true': 1, 'y_pred': 1},
   {'id': 5062, 'is_correct': True, 'y_true': 5, 'y_pred': 5},
   {'id': 8037, 'is_correct': True, 'y_true': 9, 'y_pred': 9},
   {'id': 3279, 'is_correct': True, 'y_true': 5, 'y_pred': 5},
   {'id': 6391, 'is_c